In [5]:
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg

In [6]:
conn = pg.connect(database="dse203",user="postgres", password="newPass")
news = pd.read_sql('select news.article from fuzzymatcher.news as news where \'politician\' = ANY(news.keywords) OR \'senator\' = ANY(news.keywords)', conn)
news.head()


,article
0,The freshman senator has become a surprising o...
1,WASHINGTON (Reuters) - A bipartisan group of U...
2,Republican Sen. Orrin Hatch of Utah on Tuesday...
3,WASHINGTON (Reuters) - Republican U.S. Senator...
4,"Elizabeth Warren, the Democratic senator from ..."


In [7]:
senate = pd.read_sql('select senators.name from fuzzymatcher.senate as senators', conn)
senate.head()

,name
0,Al Franken
1,Amy Klobuchar
2,Angus King
3,Ben Cardin
4,Ben Sasse


In [24]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1 # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

In [45]:
senateList = []
for senator in senators:
    first_name = senator.split()[0]
    last_name = senator.split()[-1]
    if last_name == "Jr":
         last_name = senator.split()[-2]
            
    if last_name == "III":
         last_name = senator.split()[-2]
    senateList.append(first_name + " " + last_name)
senateList

['Al Franken',
 'Amy Klobuchar',
 'Angus King',
 'Ben Cardin',
 'Ben Sasse',
 'Bernie Sanders',
 'Bill Cassidy',
 'Bill Nelson',
 'Bob Corker',
 'Bob Menendez',
 'Brian Schatz',
 'Catherine Masto',
 'Chris Coons',
 'Christopher Murphy',
 'Chris Hollen',
 'Chuck Grassley',
 'Chuck Schumer',
 'Claire McCaskill',
 'Cory Booker',
 'Cory Gardner',
 'Daniel Sullivan',
 'David Perdue',
 'Dean Heller',
 'Deb Fischer',
 'Debbie Stabenow',
 'Dianne Feinstein',
 'Dick Durbin',
 'Ed Markey',
 'Elizabeth Warren',
 'Gary Peters',
 'Heidi Heitkamp',
 'Jack Reed',
 'James Lankford',
 'Jeanne Shaheen',
 'Jeff Flake',
 'Jeff Merkley',
 'Jerry Moran',
 'Jim Inhofe',
 'Jim Risch',
 'Joe Donnelly',
 'Joe Manchin',
 'John Barrasso',
 'John Boozman',
 'John Cornyn',
 'John Hoeven',
 'John McCain',
 'John Kennedy',
 'John Thune',
 'Johnny Isakson',
 'Jon Tester',
 'Joni Ernst',
 'Kamala Harris',
 'Kirsten Gillibrand',
 'Lamar Alexander',
 'Lindsey Graham',
 'Lisa Murkowski',
 'Maggie Hassan',
 'Marco Rubio',


In [70]:
import string

def findInArticle(articleRow):
    array = news.at[articleRow,"article"]
    senators  = senate['name']
    minDist = 10000
    matches = []
    words = array.split(" ")
    prev_word = ""
    for word in words:
        table = str.maketrans({key: None for key in string.punctuation})
        word = word.translate(table)
        for senator in senateList:
            lastName = senator.split(" ")[-1]
            if((prev_word == "Senator") | (prev_word == "Sen")):
                dist = levenshtein(word, lastName)
                two_words = prev_word + " " + word
            else:
                two_words = prev_word + " " + word
                dist = levenshtein(two_words, senator)
            if dist < minDist:
                minDist = dist
                mostLikelySen = senator
        #print(two_words + "," + mostLikelySen + "," + str(minDist))
        if minDist < 2:
            matches.append(two_words + "," + mostLikelySen + "," + str(minDist))
        prev_word = word
        minDist = 10000
    #print(matches)
    return matches

In [67]:
findInArticle(0)

 The,John Thune,6
The freshman,Thad Cochran,7
freshman senator,Brian Schatz,10
senator has,Ben Cardin,8
has become,Pat Toomey,6
become a,Jack Reed,7
a surprising,Angus King,8
surprising obstacle,Orrin Hatch,12
obstacle on,Bob Casey,8
on judicial,Angus King,8
judicial nominees,Michael Bennet,11
nominees for,Jon Tester,8
for the,John Thune,6
the Trump,John Thune,6
Trump White,Tim Kaine,6
White House,Chris Hollen,7
House Louisiana,Joe Manchin,10
Louisiana GOP,Joe Manchin,10
GOP Sen,Mike Lee,6
Sen Kennedy,John Kennedy,0
Kennedy opposes,Ben Sasse,10
opposes controversial,Bernie Sanders,15
controversial Trump,Catherine Masto,14
Trump judicial,Tom Udall,9
judicial nominee

Sen,Michael Bennet,15
nominee

Sen John,Ron Johnson,12
John Kennedy,John Kennedy,0
Kennedy RLa,Ben Cardin,9
RLa said,Rand Paul,6
said Tuesday,David Perdue,7
Tuesday that,Ted Cruz,8
that he,Mike Lee,6
he would,Mike Rounds,6
would vote,Todd Young,7
vote “in,Mike Lee,6
“in a,Ben Sasse,6
a heartbeat”,Catherine Masto,9
heartbeat

The,Deb Fischer,10
time

The Louisiana,Dick Durbin,14
Louisiana senator,Brian Schatz,11
senator said,Ben Cardin,8
said he,Jack Reed,6
he would,Mike Rounds,6
would vote,Todd Young,7
vote against,Joni Ernst,7
against Talley,Chris Hollen,8
Talley “in,Angus King,8
“in a,Ben Sasse,6
a heartbeat,Ed Markey,8
heartbeat —,Dean Heller,8
— twice,Tim Kaine,6
twice if,Mike Lee,6
if I,Mike Lee,6
I can”

Kennedy,John Kennedy,7
can”

Kennedy said,John Kennedy,11
said he,Jack Reed,6
he opposed,Ben Sasse,7
opposed Katsas’,Ben Sasse,10
Katsas’ confirmation,Kamala Harris,14
confirmation to,Joni Ernst,10
to the,John Thune,6
the appellate,Dean Heller,9
appellate court,Bill Nelson,11
court in,Angus King,7
in DC,Mike Lee,6
DC because,Ben Sasse,7
because of,Ben Sasse,8
of a,Tom Udall,6
a “conflict,Marco Rubio,9
“conflict of,Jon Tester,9
of interest”,Joni Ernst,8
interest” that,Lindsey Graham,10
that the,Mike Lee,6
the senator,Jon Tester,7
senator argued,Ben Cardin,10
argued “a,Angus King,8
“a firstyear,Deb Fis

['Sen Kennedy,John Kennedy,0', 'John Kennedy,John Kennedy,0']

In [71]:
match2 = []
print(len(news.index))
for i in range(len(news.index)):
    print(i)
    match2.append(findInArticle(i))
print("---------")
flat_list = [item for sublist in match2 for item in sublist]

161
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
---------


In [72]:
flat_list

['Sen Kennedy,John Kennedy,0',
 'John Kennedy,John Kennedy,0',
 'Jeanne Shaheen,Jeanne Shaheen,0',
 'Joni Ernst,Joni Ernst,0',
 'John Thune,John Thune,0',
 'Orrin Hatch,Orrin Hatch,0',
 'Orrin Hatch,Orrin Hatch,0',
 'Senator Hatch,Orrin Hatch,0',
 'Orrin Hatch,Orrin Hatch,0',
 'John Cornyn,John Cornyn,0',
 'Elizabeth Warren,Elizabeth Warren,0',
 'Brian Schatz,Brian Schatz,0',
 'Sen Robert,Pat Roberts,1',
 'Cory Booker,Cory Booker,0',
 'Bob Menendez,Bob Menendez,0',
 'Lindsey Graham,Lindsey Graham,0',
 'Marco Rubio,Marco Rubio,0',
 'John Cornyn,John Cornyn,0',
 'Chuck Schumer,Chuck Schumer,0',
 'Mitch McConnell,Mitch McConnell,0',
 'John Cornyn,John Cornyn,0',
 'Senator McConnell,Mitch McConnell,0',
 'Lindsey Graham,Lindsey Graham,0',
 'Ron Wyden,Ron Wyden,0',
 'Lisa Murkowski,Lisa Murkowski,0',
 'Mitch McConnell,Mitch McConnell,0',
 'Lindsey Graham,Lindsey Graham,0',
 'Sheldon Whitehouse,Sheldon Whitehouse,0',
 'Senator Kennedy,John Kennedy,0',
 'Lindsey Graham,Lindsey Graham,0',
 'Lin